In [1]:
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import os, io
from surprise import SVD
from surprise import accuracy
from surprise import Reader, Dataset, SVD, evaluate
from surprise import SVDpp
from surprise import BaselineOnly

from surprise.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Reading training and testing dataset
dftrain = pd.read_csv('yelp_restaurant_train.csv')
dftest = pd.read_csv('yelp_restaurant_test.csv')

In [3]:
reader = Reader()
datatrain = Dataset.load_from_df(dftrain[['user_id', 'business_id', 'stars']], reader)

In [72]:
dftest.head(5)

,business_id,date,review_id,text,user_id,attributes,name,city,categories,avg_business_stars,avg_user_star,stars
0,fF6a_EDp0-P5Ve9Kp4WgfA,2016-08-14 01:51:06,0y8_GWd4oNpunpVEKf9R1A,The Food Good it depends on what you like comp...,vrC1d_ClcnkMqcZ7Tzsqmg,"{'WiFi': ""'no'"", 'RestaurantsTakeOut': 'True',...",Dragon Buffet,Medina,"Restaurants, Buffets",3.5,2.63,5.0
1,QnCBkwWiu19YPNnSDyF6eA,2017-10-31 15:24:01,eoEGtsGLRtlnokIWqzdtfg,I use to think that the Korean Restaurant had ...,zDKszv6zpuLsVhPFZLctXQ,"{'Caters': 'False', 'BusinessParking': ""{'gara...",Super G Mart,Charlotte,"Seafood Markets, Ethnic Food, Grocery, Food, S...",4.0,3.44,3.0
2,Q5olb1x6FGk2oLAlc9p5Lg,2017-01-21 19:18:20,0lsWHNDo-n4vP-T89mLVAA,We used yelp to find someplace for tonight and...,fe7cNj_ySXNEnpTcsNhnUA,"{'BikeParking': 'True', 'Caters': 'False', 'Bu...",Bonefish Grill,Las Vegas,"Seafood, Restaurants, Nightlife, American (New...",3.5,4.66,5.0
3,UkFMV7WGFfE_u8zyUkkuqg,2015-08-15 18:14:16,Z8wiunFnj9SbtlVGhbeBmg,Went here with my son busy place I knew it had...,M2ywRTY1UYTiJEDp2x14yQ,"{'RestaurantsPriceRange2': '1', 'BusinessParki...",La Flor de Michoacan,Las Vegas,"Ice Cream & Frozen Yogurt, Juice Bars & Smooth...",4.0,3.31,4.0
4,GdnLAgRXt43eISMtJEOZRg,2013-05-31 04:00:37,wHr1eE2WAiFsTijXX0F0kw,I was excited to try this place out and then Y...,BS3qOK1iRLcWtoV9KnwyXg,"{'HasTV': 'True', 'RestaurantsReservations': '...",Sushi Express,Madison,"Restaurants, Japanese, Sushi Bars",4.0,3.07,4.0


In [5]:
bsl_options = {'method': 'als',
               'n_epochs': 7, 
               'reg_u': 3, 
               'reg_i': 7   
               }

In [6]:
trainset, testset = train_test_split(datatrain, test_size=0.20)

# Applying Baseline model

In [7]:
algo = BaselineOnly(bsl_options=bsl_options)
predictions_baseline = algo.fit(trainset).test(testset)

Estimating biases using als...


In [8]:
accuracy.rmse(predictions_baseline)

RMSE: 1.0793


1.0793281863598205

In [9]:
cross_validate(algo, datatrain, measures=['RMSE'], cv=5, verbose = False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.0780467 , 1.07734159, 1.0804667 , 1.07897398, 1.0791776 ]),
 'fit_time': (3.30057692527771,
  3.9825031757354736,
  3.867738723754883,
  3.7335381507873535,
  3.876842975616455),
 'test_time': (2.396864175796509,
  2.062152862548828,
  1.9600298404693604,
  1.8828670978546143,
  2.6509549617767334)}

In [10]:
len(dftest)

336980

In [11]:
ui = np.array(dftest['user_id'])
utest = ui.tolist() 

bi = np.array(dftest['business_id'])
btest = bi.tolist()

In [12]:
#Predicting the ratings using Baseline model 
predbaseline = []

for item in range(len(dftest)):
    predbaseline.append(round(algo.predict(utest[item], btest[item])[3], 2)) #appending pred list 

In [13]:
len(predbaseline)

336980

In [23]:
from collections import defaultdict
 
def get_top_recommendations(predictions, topN = 5):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [24]:
top_recommendations = get_top_recommendations(predictions_baseline)

In [25]:
# For each user we print recommendation
i=0;
item_list = []
for uid, user_ratings in top_recommendations.items():
    print("user", uid)
    for (iid, _) in user_ratings:
        item_list.append(iid)
        print("item", iid)
    i=i+1;
    if(i==1):
        break;

user ayS9T8DTm2xV5mrsJIsIrQ
item R9bFr7dADF2fosWJjgbyWQ
item cj_9eTPR0JBqk9qt2VmzFw
item Aov96CM4FZAXeZvKtsStdA
item ABRgXNwdOX_JyqChNr8IYw


In [71]:
for i in range(len(dftest)):
    if dftest['user_id'][i] == 'ayS9T8DTm2xV5mrsJIsIrQ':
        print("User ID: 'ayS9T8DTm2xV5mrsJIsIrQ' Restaurant: ",dftest[dftest['business_id'].isin(item_list)]['name'].unique())
        break

User ID: 'ayS9T8DTm2xV5mrsJIsIrQ' Restaurant:  ['Swatow Restaurant' 'Hodo Kwaja' 'Que Ling Restaurant'
 'Nawzar Meat Market']


# Applying SVD

In [23]:
svd = SVD()
evaluate(svd, datatrain, measures=['RMSE'])

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 1.0880
------------
Fold 2
RMSE: 1.0880
------------
Fold 3
RMSE: 1.0876
------------
Fold 4
RMSE: 1.0935
------------
Fold 5
RMSE: 1.0888
------------
------------
Mean RMSE: 1.0892
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.087956748509602,
                             1.0880179466707063,
                             1.0875660672119771,
                             1.093451337618312,
                             1.0888466411462412]})

In [24]:
svd.train(trainset)

In [25]:
predictions_svd = svd.test(testset)

In [26]:
accuracy.rmse(predictions_svd, verbose=True)

RMSE: 1.0885


1.0884932893244363

In [27]:
#Predicting the ratings using SVD model 
predsvd = []

for item in range(len(dftest)):
    predsvd.append(round(svd.predict(utest[item], btest[item])[3], 2)) #appending pred list 

# Applying SVDpp

In [30]:
algosvdpp = SVDpp(n_factors=150,n_epochs=40)

In [31]:
algosvdpp.fit(trainset)

In [32]:
predictions_svdpp = algosvdpp.test(testset)

In [33]:
accuracy.rmse(predictions_svdpp, verbose=True)

RMSE: 1.1172


1.1171530768175075

In [34]:
predsvdpp = []

for item in range(len(dftest)):
    predsvdpp.append(round(algosvdpp.predict(utest[item], btest[item])[3], 2)) #appending pred list 

In [38]:
#Saving the stars in csv file
with open('baseline.csv','w') as filehandle:
    for listitem in predbaseline:
        filehandle.write('%0.2f\n' % listitem)

In [39]:
with open('svd.csv','w') as filehandle:
    for listitem in predsvd:
        filehandle.write('%0.2f\n' % listitem)

In [40]:
with open('svdpp.csv','w') as filehandle:
    for listitem in predsvdpp:
        filehandle.write('%0.2f\n' % listitem)

# Grid Search

In [1]:
from surprise.model_selection import GridSearchCV

In [2]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005,0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[10,50]}

In [5]:
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)

In [8]:
grid_search.fit(datatrain)
#Took a lot of time to train, so skip Grid Search model.